In [1]:
import numpy as np
import pandas as pd
import pickle
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import os 
from functions import *

# Load dataset

In [31]:
S = pd.read_csv('../data/IMdata/S.csv')
S = pd.read_csv('../data/IMdata/S.csv',index_col=0)
A = pd.read_csv('../data/IMdata/A.csv',index_col=0)
log_rpkm_norm = pd.read_csv('../data/IMdata/log_rpkm_norm.csv', index_col=0)
regulons = load_pickle('../data/IMdata/regulons.pkl')
IM_list = list(S.columns)[1:]

In [29]:
orig_var = (log_rpkm_norm**2).sum().sum()
err_var = ((log_rpkm_norm.values - (S.values).dot(A.values) )**2).sum().sum()
print('Explained variance: {:.2f}'.format(1-err_var/orig_var))

Explained variance: 0.85


# IM gene identification

In [22]:
IM_thersholds = {}
for im in IM_list:
    IM_thersholds[im] = compute_threshold(S, im ,cutoff=550)

# Regulon enrichment